In [1]:
from langgraph.graph import START, END, StateGraph
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from typing import TypedDict, Annotated, Sequence
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph.message import add_messages
from langchain.tools.retriever import create_retriever_tool
from langgraph.prebuilt import ToolNode, tools_condition
from langchain import hub
from IPython.display import Image, display
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
llm=ChatGroq(model_name="Gemma2-9b-It")
embedding = OllamaEmbeddings(model = "nomic-embed-text")

In [3]:
loader = WebBaseLoader(web_path="https://learn.microsoft.com/en-us/azure/ai-services/computer-vision/overview-identity")
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = splitter.split_documents(docs)
docs

[Document(metadata={'source': 'https://learn.microsoft.com/en-us/azure/ai-services/computer-vision/overview-identity', 'title': 'What is the Azure AI Face service? - Azure AI services | Microsoft Learn', 'description': 'The Azure AI Face service provides AI algorithms that you use to detect, recognize, and analyze human faces in images.', 'language': 'en-us'}, page_content='What is the Azure AI Face service? - Azure AI services | Microsoft Learn\n\n\n\n\nSkip to main content\n\n\nThis browser is no longer supported.\nUpgrade to Microsoft Edge to take advantage of the latest features, security updates, and technical support.\n\nDownload Microsoft Edge\nMore info about Internet Explorer and Microsoft Edge\n\n\n\n\n\n\n\n\n\n\n\n\n\nTable of contents\n\n\n\nExit focus mode\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRead in English\n\n\n\n\nSave\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nTable of contents\n\n\n\nRead in English\n\n\n\n\nSave\n\n\n\n\n\nAdd to plan\n\n\n\n\nEdit\n\n\nShare 

In [4]:
vector_store = Chroma.from_documents(
    collection_name = "agentic_rag", embedding=embedding, documents=docs
)

retriever = vector_store.as_retriever()

retriever_tool=create_retriever_tool(
    retriever,
    "retrieve_blog_posts",
    "Search and return information about Azure fce services.You are a specialized assistant. Use the 'retriever_tool' **only** when the query explicitly relates to LangChain blog data. For all other queries, respond directly without using any tool. For simple queries like 'hi', 'hello', or 'how are you', provide a normal response.",
    )

tools = [retriever_tool]

In [5]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]


class DocumentGrader(BaseModel):
    score:bool =  Field(description="A binary score, which is True if question is related to the given information, otherwise False")

In [ ]:
prompt_template = ChatPromptTemplate.from_template(
    [
        SystemMessage(
        """
            You are an AI assistant. You check relevance of a documents w.r.t given user question.
            If the documnet's internal meaning is close to that of the question, consider the document as relevant and return True.
            Otherwise False
        """),
        ("human", "Retrieved Document: {documents} \n Question {question}")
    ]
)
llm_with_strucutred_output = llm.with_structured_output(DocumentGrader)
documents = retriever.get_relevant_documents()
chain = prompt_template | llm_with_strucutred_output


response = chain.invoke({"question" : question, "documents" : documents})

In [ ]:
wf = StateGraph(AgentState)

wf.add_node("Docs_Vector_Retrieve", retrieve)
wf.add_node("Retrieved_Documnets_Relevance", documnets_relevance)
wf.add_node("Content_Generator", generate)
wf.add_node("Transform_User_Query", transform_query)

wf.add_edge(START, "Docs_Vector_Retrieve")
wf.add_edge("Docs_Vector_Retrieve", "Retrieved_Documnets_Relevance")
wf.add_conditional_edges("Retrieved_Documnets_Relevance", decide_to_generate, {'generate': "Content_Generator", 'transform': "Transform_User_Query"})
wf.add_conditional_edges("Content_Generato", rewrite_query, {'yes': 'Transform_Query', 'no': END})
wf.add_conditional_edges("Transform_Query", retrieve_after_transformation, {"retrieve": "Docs_Vector_Retrieve", "irrelevant": END})

app = wf.compile()